# 🚩 문제지 (Mission Brief)

## 🏕️ 스토리: “캠핑 마스터 AI”
국내 최대 아웃도어 기업 **캠핑 마스터**는 △장비 추천 챗봇 △캠핑장 리뷰 분석 △사전 안전 점검 알람 등을 제공하는 **통합 AI 어시스턴트**를 출시하려 합니다. 여러분은 막 입사한 **NLP 엔지니어**로서, 다음 네 단계의 모듈을 순차적으로 구축‧실험해야 합니다.

각 단계의 **starter code**는 50 % 이상 완성돼 있으니, `### TODO` 표시 부분만 채워서 실행·검증하세요.

> **환경 준비** (필수)  
> ```bash
> pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
> pip install datasets transformers accelerate seqeval
> ```


### 과제 1 (의도 분류 GRU)  
**목표** : 사용자의 자연어 요청을 150 개 의도로 분류해 챗봇 라우팅 성능을 평가한다.  
**데이터** : Hugging Face → `clinc_oos`, config `"small"`  

```python
from datasets import load_dataset
raw = load_dataset("clinc_oos", "small")
print(raw["train"][0])
```
#### Starter Code


In [ ]:
! pip install transformers datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

MAX_LEN, BATCH = 32, 64
tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")
pad_id = tok.pad_token_id

def tokenize(b):
    return tok(b["text"], padding="max_length",
               truncation=True, max_length=MAX_LEN)

ds = load_dataset("clinc_oos", "small")

# 컬럼명 intent → labels
ds = ds.rename_column("intent", "labels")

# 토크나이즈
ds = ds.map(tokenize, batched=True)

# Torch 텐서 형식으로 지정
ds.set_format(type="torch",
              columns=["input_ids", "attention_mask", "labels"])

train_dl = DataLoader(ds["train"], batch_size=BATCH, shuffle=True)
test_dl  = DataLoader(ds["test"],  batch_size=BATCH)


class GRUClassifier(nn.Module):
    def __init__(self, vocab, embed_dim, hidden, num_labels):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim, padding_idx=tokenizer.pad_token_id)
        self.gru   = nn.GRU(embed_dim, hidden, batch_first=True, bidirectional=True)
        self.fc    = nn.Linear(hidden*2, num_labels)

    def forward(self, ids, mask):
        x = self.embed(ids)
        x, _ = self.gru(x)
        x = torch.cat([x[:,0,:hidden], x[:,-1,hidden:]], dim=-1)  # 양방향 첫·끝 스텝 결합
        return self.fc(x)

### TODO 1-a : 임베딩 차원, hidden size 등 하이퍼파라미터 지정
### TODO 1-b : 훈련 루프(optimizer, loss 정의 및 epoch 반복) 작성
### TODO 1-c : 테스트 셋 정확도 출력


---

### 과제 2 (BiLSTM + CRF NE 인식)  
**목표** : 캠핑 준비물 체크리스트에서 **장비·지역·날짜** 등 엔티티를 추출한다.  
**데이터** : `conll2003` (영문) – 실제 캠핑 도메인과 다르지만, 모델 아키텍처 학습용으로 사용.  

> CRF 계층 구현이 부담스러우면, `seqeval` 평가 지표와 **BiLSTM+Linear** 구조만 완성해도 통과점수를 얻을 수 있다.

```python
# 데이터 예시
from datasets import load_dataset
ner = load_dataset("conll2003")
print(ner["train"][0])
```
#### Starter Code (BiLSTM Tagger skeleton)


In [3]:
class BiLSTMTagger(nn.Module):
    def __init__(self, vocab, tagset, embed_dim=100, hidden=256):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim)
        self.lstm  = nn.LSTM(embed_dim, hidden, num_layers=1,
                             batch_first=True, bidirectional=True)
        self.fc    = nn.Linear(hidden*2, tagset)

    def forward(self, ids, mask):
        em = self.embed(ids)
        out, _ = self.lstm(em)
        return self.fc(out)          # [batch, seq, tag]

### TODO 2-a : 토크나이저를 훈련 셋 vocab 으로 학습
### TODO 2-b : padding & masking 함수 구현
### TODO 2-c : CrossEntropyLoss(ignore_index=pad_tag) 로 모델 학습
### TODO 2-d : seqeval (f1) 로 성능 측정


---

### 과제 3 (1D CNN 리뷰 감성 분류)  
**목표** : 캠핑장 리뷰를 긍정/부정으로 분류하여 별점 이상/이하를 탐지한다.  
**데이터** : Hugging Face → `imdb` (binary sentiment)

#### Starter Code


In [6]:
class TextCNN(nn.Module):
    def __init__(self, vocab, embed_dim=128, num_classes=2, k=[3,4,5]):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim, padding_idx=tokenizer.pad_token_id)
        self.convs = nn.ModuleList(
            [nn.Conv1d(embed_dim, 100, kernel_size=ks) for ks in k])
        self.fc = nn.Linear(100*len(k), num_classes)

    def forward(self, ids):
        x = self.embed(ids).transpose(1,2)                # [B,C,L]
        x = torch.cat([torch.relu(c(x)).max(2)[0] for c in self.convs], 1)
        return self.fc(x)

### TODO 3-a : 토큰화·DataLoader 작성
### TODO 3-b : 훈련 루프 및 정확도 평가


## 데이터셋 다운로드 요약

| 과제 | 데이터셋 (🤗 Datasets) | 호출 예시 |
|------|-----------------------|-----------|
| 1 | `clinc_oos`, `"small"` | `load_dataset("clinc_oos","small")` |
| 2 | `conll2003` | `load_dataset("conll2003")` |
| 3 | `imdb` | `load_dataset("imdb")` |
| 4 | `daily_dialog` | `load_dataset("daily_dialog")` |

### 마무리 팁
- 각 과제 스크립트는 `torch.save(model.state_dict(), "...")`로 체크포인트를 저장해두면, 후속 실험(하이퍼파라미터 튜닝, 앙상블) 시 시간을 절약할 수 있습니다.  
- CPU 에서 빠르게 실습하려면 `datasets` 의 `.select(range(N))`로 샘플 수를 줄여 디버깅 후, GPU 훈련 환경에서 전체 데이터로 재훈련하세요.

성공적인 **캠핑 마스터 AI** 출시를 기원합니다! 🏕️